In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from functools import partial
from typing import Callable, List, Tuple, NoReturn, Union, Dict
import warnings
import pickle
import sys

from torchmetrics import F1Score
import albumentations as A
import pandas as pd
import numpy as np
import cv2
import torch
import torchvision
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import vidaug.augmentors as va
import onnxruntime as ort

sys.path.append('../src/')
from avg_video import avg_video
from dataset import AvgVideoDataModule
from model import MobileNetV3
from utils import save2onnx

warnings.filterwarnings("ignore")

2023-06-02 20:16:47.335540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 20:16:48.503546: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
DATA_DIR = Path('../data/')
SRC_DIR = Path('../src/')
SUBMIT_PATH = Path('../output/submit_avg_video/')
CLASSES = ["bridge_down", "bridge_up", "no_action", "train_in_out"]
SEED = 0
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
with open(SRC_DIR.joinpath('id2label.pkl'), 'rb') as fp:
    id2label = pickle.load(fp)
    label2id = {v: k for k, v in id2label.items()}

In [5]:
np.random.seed(SEED)
torch.manual_seed(SEED)
pl.seed_everything(SEED);

Global seed set to 0


# Data

In [6]:
train_clips_avg = sum([list(DATA_DIR.joinpath("train_avg", c).glob("*.jpg")) for c in CLASSES], [])
train_clips_avg = pd.DataFrame([clip.parts[-2:] for clip in train_clips_avg], columns=["label", "fname"])

train_clips_avg

,label,fname
0,bridge_down,d6be86768fa129b8.jpg
1,bridge_down,1b98537880b065ff.jpg
2,bridge_down,462ce800d75fb407.jpg
3,bridge_down,92848c13f18c8442.jpg
4,bridge_down,928cc9eaf2eb7590.jpg
...,...,...
491,train_in_out,7b11b87dd010c638.jpg
492,train_in_out,e0351f88ed52db65.jpg
493,train_in_out,32afa0ba3020e09f.jpg
494,train_in_out,5e53e235164e5386.jpg


In [8]:
train_data, val_data = train_test_split(train_clips_avg,
                                        train_size=0.3,
                                        stratify=train_clips_avg['label'],
                                        shuffle=True,
                                        random_state=SEED)

train_labels = train_data['label'].map(id2label)
val_labels = val_data['label'].map(label2id)

In [11]:
datamodule = AvgVideoDataModule(train_clips_avg, train_clips_avg)
model = MobileNetV3()

In [12]:
early_stopping = EarlyStopping(monitor='loss_val',
                               mode='min',
                               patience=3)

checkpoint = ModelCheckpoint(monitor='loss_val',
                             mode='min',
                             dirpath=SUBMIT_PATH,
                             filename='mnv3__{epoch}-{f1_val:.2f}',
                             save_top_k=1)

trainer = pl.Trainer(
    max_epochs=10,
    accelerator='gpu',
    devices=[0],
    callbacks=[early_stopping, checkpoint],
    logger=False
)

In [14]:
trainer.fit(model, datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name     | Type              | Params
-----------------------------------------------
0 | model    | MobileNetV3       | 1.5 M 
1 | f1_train | MulticlassF1Score | 0     
2 | f1_val   | MulticlassF1Score | 0     
3 | loss_fn  | CrossEntropyLoss  | 0     
-----------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.088     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [15]:
save2onnx(model.model, SUBMIT_PATH.joinpath('mnv3.onnx'))

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

